In [ ]:
import os, sys
sys.path.append('..')
from os.path import abspath, dirname
import zarr
import z5py
import numpy as np
import pandas as pd
from glob import glob 
from skimage.measure import regionprops
from skimage.io import imread, imsave
from scipy import stats
from scipy.stats import skewnorm, lognorm
from scipy.optimize import minimize
import itertools
import seaborn as sns

from easi_fish import n5_metadata_utils as n5mu
from easi_fish import roi_prop, spot, intensity
import warnings
warnings.filterwarnings('ignore')

import importlib
importlib.reload(spot)
importlib.reload(roi_prop)
importlib.reload(intensity)

"""
Spot counts for cells with highly expressed genes (dense spots)
1. Measure total intensity of every ROI after bleed-through correction and background subtraction.
2. Calculate the number of spot from total intensity based on unit-spot intensity
3. Correlate the number of spots (from air-localize) with the total fluorescence intensity/voxel in each ROI and determine a 'cutoff'. 
   Spot count > cutoff: use spot count converted based on total fluorescence intensity; 
   Spot count < cutoff: use spot count from Airlocalize
"""

In [ ]:
## input
input_dir = "D:\\SWAP\\Vincent\\lt171_FlpO\\gene_new_4tile\\outputs"
output_dir = os.path.join(input_dir, "testfull-aug9")  

# fixed image (directory - n5 format)
fix_dir = os.path.join(input_dir, "r2\\export_sigma3.n5")
# get appropriate image data within fix_dir
subpath='\\c3\\s2'

# registered image (directory - n5 format)
reg_dir = os.path.join(input_dir, "r1\\registration\\lt171_gene_4tile_r1-to-lt171_gene_4tile_r2\\warped")  

# segmentation mask (tif format accepted here)
lb_dir  = os.path.join(input_dir, "r2\\segmentation\\lt171_gene_4tile_r2-c3.tif" ) 

# spot dir 
spot_dir = os.path.join(input_dir, "spots_pooled") # pool spots together; warpped and fixed
intn_dir = os.path.join(input_dir, "intensities_pooled") # pool spots together; warpped and fixed
# for every gene
rounds = ['r1', 'r2']
channels = ['c0', 'c1', 'c2', 'c4']
# r1 should be the wrappped one
fx_spots = [os.path.join(spot_dir, f'spots_{r}_{c}.txt') 
                 for r, c in itertools.product(rounds, channels)]
fx_intns = [os.path.join(intn_dir, f'{r}_{c}_intensity.csv') 
                 for r, c in itertools.product(rounds, channels)]

for f in fx_spots:
    assert os.path.isfile(f)
for f in fx_intns:
    assert os.path.isfile(f)

## output
out_mask = os.path.join(output_dir, 'mask.tif')
out_badroi = os.path.join(output_dir, 'bad_roi_list.npy')
out_allroi = os.path.join(output_dir, "roi_all.csv") 
out_roi = os.path.join(output_dir, "roi.csv") 
out_spots = os.path.join(output_dir, "spotcount.csv")
out_spots_intn = os.path.join(output_dir, "spotcount_intn.csv")
# out_intensity = os.path.join(output_dir, "intensity_c0_r2.csv")
out_spots_merged = os.path.join(output_dir, 'spotcount_merged.csv')

In [ ]:
lb=imread(lb_dir)
print(lb.shape)
roi = np.max(lb)
print(roi)

In [ ]:
spots = np.loadtxt(fx_spots[0], delimiter=',')
print(spots.shape)
spots

# check spots range

In [ ]:
print(np.nanmin(spots[:,0]), np.nanmax(spots[:,0]))
print(np.nanmin(spots[:,1]), np.nanmax(spots[:,1]))
print(np.nanmin(spots[:,2]), np.nanmax(spots[:,2]))

In [ ]:
sns.boxplot(data=spots[:,:3])

In [ ]:
sns.boxplot(data=spots[:,:3], showfliers=False)

In [ ]:
sns.violinplot(data=spots[:,:3], showfliers=False)

# test one channel

In [ ]:
%%time
importlib.reload(spot)
spotcount = spot.spot_counts(lb, spots, verbose=False)
res0 = spotcount['spot count'] #.values

In [ ]:
%%time
importlib.reload(spot)
res = spot.spot_counts_worker(lb, spots)

In [ ]:
cond = (res == res0)
res[~cond], res0[~cond]

In [ ]:
# discrepany only happens at the edge
allroi = pd.read_csv(out_allroi).set_index('roi')
allroi.loc[~cond]

# test all channels
- only differences are on the edge

In [ ]:
%%time
# apply to all channels
importlib.reload(spot)
resall = spot.spot_counts_v2(lb, spot_dir, s=[0.92,0.92,0.84], verbose=True)

In [ ]:
resall

In [ ]:
%%time
# apply to all channels
importlib.reload(spot)
resallv0 = spot.spot_counts(lb, spot_dir, s=[0.92,0.92,0.84], verbose=False)

In [ ]:
resallv0

In [ ]:
condall = resallv0 == resall
(~condall).sum()

In [ ]:
for col in condall:
    print(resallv0.loc[~condall[col], col] - resall.loc[~condall[col], col])
    print(allroi.loc[~condall[col]])